# 거래소에서 option data 가져와보기

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import time
from datetime import datetime, timedelta

from typing import List, Tuple 
from tqdm import tqdm

import requests

In [2]:
## custom libs

from krx_config import API_URL, HEADERS, PAYLOAD_TEMPLATE

In [3]:
def generate_date_range(start_date: str, end_date: str) -> List[str]:
    """
    Generates a list of dates in YYYYMMDD format between start_date and end_date inclusive.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        List[str]: List of dates as strings in 'YYYYMMDD' format.
    """
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = end - start
    return [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range(delta.days + 1)]

def fetch_option_data(session: requests.Session, trade_date: str) -> pd.DataFrame:
    """
    Fetches option data for a specific trade date.

    Args:
        session (requests.Session): The requests session with headers set.
        trade_date (str): Trade date in 'YYYYMMDD' format.

    Returns:
        pd.DataFrame: DataFrame containing option data for the trade date.
    """
    payload = PAYLOAD_TEMPLATE.copy()
    payload["trdDd"] = trade_date

    response = session.post(API_URL, data=payload)
    response.raise_for_status()

    data = response.json()

    if "output" not in data:
        return pd.DataFrame()

    return pd.DataFrame(data["output"])

def scrape_krx_option_data(start_date: str, end_date: str) -> pd.DataFrame:
    """
    Scrapes KRX option data between start_date and end_date.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: Combined DataFrame containing option data for all dates.
    """
    dates = generate_date_range(start_date, end_date)
    all_data = []

    with requests.Session() as session:
        session.headers.update(HEADERS)
        for date in tqdm(dates, desc="Fetching data"):
            try:
                daily_data = fetch_option_data(session, date)
                if not daily_data.empty:
                    daily_data['Trade_Date'] = datetime.strptime(date, "%Y%m%d").date()
                    all_data.append(daily_data)
                time.sleep(1)  # Delay to mimic human behavior
            except requests.HTTPError as http_err:
                print(f"HTTP error for date {date}: {http_err}")
            except Exception as err:
                print(f"Error for date {date}: {err}")

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

def save_to_excel(df: pd.DataFrame, filename: str):
    """
    Saves the DataFrame to an Excel file.

    Args:
        df (pd.DataFrame): DataFrame to save.
        filename (str): Filename for the Excel file.
    """
    df.to_excel(filename, index=False)
    print(f"Data saved to {filename}")

In [4]:
sample_date = '20241204'

In [5]:
with requests.Session() as session:
    session.headers.update(HEADERS)
    payload = PAYLOAD_TEMPLATE.copy()
    payload["trdDd"] = sample_date

    response = session.post(API_URL, data=payload)

: 

: 

In [ ]:
dd = response.json()

In [21]:
START_DATE = '2021-01-01'
END_DATE = '2024-12-05'

In [ ]:
option_data_df = scrape_krx_option_data(START_DATE, END_DATE)

Fetching data:  37%|███▋      | 524/1435 [37:09<5:12:52, 20.61s/it]

HTTP error for date 20220608: 400 Client Error: Bad Request for url: http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd


Fetching data:  37%|███▋      | 525/1435 [38:10<8:16:21, 32.73s/it]

HTTP error for date 20220609: 400 Client Error: Bad Request for url: http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd


Fetching data:  37%|███▋      | 526/1435 [39:11<10:24:23, 41.21s/it]

...

HTTP error for date 20220825: 400 Client Error: Bad Request for url: http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd


Fetching data:  42%|████▏     | 603/1435 [1:34:31<12:43:04, 55.03s/it]

HTTP error for date 20220826: 400 Client Error: Bad Request for url: http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd


Fetching data:  42%|████▏     | 606/1435 [1:35:34<8:37:14, 37.44s/it] 

HTTP error for date 20220829: 400 Client Error: Bad Request for url: http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd
